In [29]:
%%capture
!pip install scholarly
!pip install fuzzywuzzy[speedup]

In [1]:
import scholarly
from fuzzywuzzy import process
from multiprocessing import Pool
import worker

In [2]:
%%time
# Retrieve the author's data, fill-in, and print
search_query = scholarly.search_author('Zachary F M Burton')
author = next(search_query)
author = author.fill()
print(author)

{'_filled': True,
 'affiliation': 'Ph.D. Candidate, Stanford University',
 'citedby': 13,
 'citedby5y': 13,
 'cites_per_year': {2019: 8, 2020: 5},
 'coauthors': [<scholarly.scholarly.Author object at 0x0000020BFFC91DA0>,
 'email': '@stanford.edu',
 'hindex': 2,
 'hindex5y': 2,
 'i10index': 0,
 'i10index5y': 0,
 'id': 'X7FY3wUAAAAJ',
 'interests': ['Gas Hydrates',
               'Geochemistry',
               'Planetary Science',
               'Basin Modeling',
               'Sedimentary Systems'],
 'name': 'Zachary F. M. Burton',
 'publications': [<scholarly.scholarly.Publication object at 0x0000020BFFC91F28>,
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=X7FY3wUAAAAJ'}
Wall time: 22.3 s


In [3]:
publications = author.publications

In [4]:
publications[0].bib['title']

'Unraveling petroleum degradation, maturity, and mixing and addressing impact on petroleum prospectivity: Insights from frontier exploration regions in New Zealand'

In [5]:
%%time
pubs = [p.bib['title'] for p in publications]

Wall time: 0 ns


In [17]:
%%time
pubs_info = [next(scholarly.search_pubs_query(p)) for p in pubs]

Wall time: 2min 9s


In [7]:
len(pubs)

17

In [8]:
%%time 
if __name__ == "__main__":
    pool = Pool()
    pubs_info = pool.map(worker.get_pub_info, pubs)

Wall time: 24.7 s


In [11]:
len(pubs_info)

8

In [13]:
if pubs_info[0]: pubs_info[0].bib['author']
else: print('none')

none


In [10]:
pubs_authors = [p.bib['author'] for p in pubs_info if p]
len(pubs_authors)

0

In [12]:
pubs_authors

['ZFM Burton and JM Moldowan and LB Magoon…',
 'JL Bishop and JD Toner and P Englert…',
 'ZFM Burton and JL Bishop and JM Danielsen']

In [13]:
author_name = 'Zachary F M Burton'
process.extractOne(author_name, pubs_authors[0].split(' and '))

('ZFM Burton', 86)

In [14]:
pub_authors_lists = [authors.split(' and ') for authors in pubs_authors]

In [15]:
author_positions = {}
for lis in pub_authors_lists:
    match = process.extractOne(author_name, lis)[0]
    for i, author in enumerate(lis):
        if author == match:
            if i+1 in author_positions:
                author_positions[i+1] += 1
            else:
                author_positions[i+1] = 1
            break
            
author_positions 

{1: 2, 2: 1}

In [16]:
# TODO: 
    # use multiprocessing is only getting a few of the pubs, and None for the rest
    # I think it's a timeout error
    
    # get citations & h-index based on author position (A1: 4, A2: 7, etc.)
    # make nice dashboard w/ simple histograms